In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.dates as dates
import pandas_profiling
#import xgboost as xgb
import pickle

from matplotlib.gridspec import GridSpec
from datetime import datetime, date, time, timedelta
from sklearn.datasets import make_classification
from sklearn.preprocessing import StandardScaler, OneHotEncoder, KBinsDiscretizer, MinMaxScaler, QuantileTransformer
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import TimeSeriesSplit #Para split temporal
from sklearn.model_selection import GridSearchCV
#from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, confusion_matrix, auc, roc_curve

In [ ]:
#import functions created in .py
import load_data 
import clean_data
import transform_data
import eda
import geda
import feature_selection

# 1. Introducción

Este proyecto esta enfocado a predecir si una mujer es o no mayor a 23 años dado un conjunto de características presentado en una base de datos. 

La base de datos corresponde a las [*Interrupciones Legales de Embarazo*](https://datos.cdmx.gob.mx/explore/dataset/interrupcion-legal-del-embarazo) en la Ciudad de México. Esta base se obtuvo de la página de datos abiertos de la ciudad. 

Para contextualizar este proyecto, se consideran los siguientes supuestos:

  - El resultado de este proyecto será utilizado por una institución gubernamental, por lo tanto, los recursos son finitos, en particular se tiene un presupuesto limitado.
  - Se planea lanzar una campaña de servicios médicos para las mujeres que abortan. Se desea ofrecer servicios a bajo costo o gratuitos de salpingoclasia (esterilización permanente) y se quiere estimular a esta mujeres a participar en otros programas de bienestar social a los que pueden ser elegibles (por ejemplo, de madres solteras)

## 1.1. Interrupción legal del embarazo en la Ciudad de México

Es importante mencionar que la Interrupción del Embarazo solo era legal en la Ciudad de México, hasta hace unas pocas semanas, sin importar la causa o motivo. Pues si bien en otros estados también esta permitido, es sólo bajo ciertas circunstancias, principalmente relacionadas con delitos sexuales.

La interrupción del embarazo es legal en la Ciudad de México hasta las 12 semanas de gestación. Actualmente existen 13 clínicas en la ciudad que otorgan este servicio. La página de información de la CDMX menciona que los servicios se **proporcionan de manera legal, segura, confidencial y gratuita**. 

Los requisitos oficiales para obtener el servicio son los siguientes.

Requisitos para residentes de la Ciudad de México
+ Identificación oficial, en original y copia.
+ Comprobante de domicilio (último recibo de predial, luz, agua, gas, televisión de paga, teléfono fijo o servicio de internet), en original y copia.
+ Hoja de Gratuidad. Una trabajadora social te ayudará en caso de no tenerla.
+ Un acompañante con identificación oficial en original y copia.
+ De manera opcional en los hospitales pueden solicitarte: CURP y/o acta de nacimiento.

Requisitos para menores de edad:
+ Acta de Nacimiento en original y copia.
+ CURP (Puede imprimirse desde este sitio web)
+ Credencial o documento con fotografía reciente (credencial de la escuela o certificado de estudios) en original y copia.
+ Comprobante de domicilio en original y copia (último recibo de predial, luz, agua, gas, televisión de paga, teléfono fijo o servicio de internet).
+ Acudir acompañada por madre, padre, tutor o representante legal con identificación oficial y comprobante de domicilio, ambos en original y copia.

Requisitos para residentes de otros estados:
+ Original y copia de identificación oficial.
+ Comprobante de domicilio en original y copia.
+ Un acompañante con identificación oficial en original y copia.

# 2. Preprocesamiento de la base

In [ ]:
##################################################  CARGAR BASE  #############################################################
#Update data file name
datafile_name = 'interrupcion-legal-del-embarazo.csv'
#Update path to data file

#path_to_datafile = '~/Documents/Brenda/Proyecto/Data'
path_to_datafile = '/home/bj/Documents/IntroDataScience/Proyecto/Data'
#path_to_datafile ='C:/Users/GZARAZUA/Desktop/MCD/MINERIA DE DATOS/Reporte aborto'

#we load the database
interrupcion_legal = load_data.load_df(datafile_name, path_to_datafile, ';')

## 2.1. Descripción de la base

La base contiene información de interrupciones legales de embarazo de 2016 a 2019
Observamos que las variables son en su mayoría numericas (conteos) y algunas categóricas, que se relacionan en lo general con lo siguiente:
+ Fecha de registro: ano, mes, fingreso
+ Sitio donde se atiende al paciente y datos relacionados a la intervención: cve_hospital, autoref, desc_derechohab, consejería, h_ingreso, desc_servicio, p_consent, procile, s_complica, panticoncept
+ Características propias de la persona: edocivil, edad, nivel_edu, ocupacion, religion, parentezco, entidad, alc_o_municipio, menarca, fsexual, fmenstrua, sememb, nhijos, gesta, naborto, npartos, ncesarea, nile, anticonceptivo, c_num, motiles, p_semgest, p_diasgesta

Esta base contiene información acerca de los procedimientos de interrupción legal del embarazo (ILE) que se realizaron en las clínicas y hospitales de la Secretaría de Salud de la Ciudad de México a partir de 2016.

Diccionario de datos:
+ ano: Año en que se realizó el procedimiento
+ mes: Mes en que se realizó el procedimiento
+ cve_hospital: Clave de la Unidad Médica donde se realizó el procedimiento
+ fingreso: Fecha de Interrupción Legal del Embarazo
+ autoref: Especificar si la paciente es referida de otra unidad o acude directamente
+ edocivil_descripcion: Estado civil
+ edad: Edad cumplida en años
+ desc_derechohab: Especificar la institución que otorga la derechohabiencia
+ nivel_edu: Último nivel escola acreditado
+ ocupacion: Ocupación
+ religion: Religión
+ parentesco: Parentesco del responsable con la paciente
+ entidad: Entidad de residencia
+ alc_o_municipio: Delegación o municipio de residencia
+ menarca: Edad de la primera menstruación
+ fsexual: Edad de inicio de vida sexual activa
+ fmenstrua: Fecha de la última menstruación
+ sememb: Semanas de embarazo por fecha de última menstruación
+ nhijos: Número de hijos
+ gesta: Número de embarazos (Incluyendo abortos)
+ naborto: Número de abortos
+ npartos: Número de nacimientos por vía vaginal
+ ncesarea: Número de cesáreas
+ nile: Número de Interrupciones Legales del Embarazo previas
+ consejeria: Especificar si la paciente recibió consejería sobre la ILE
+ anticonceptivo: Especificar si la paciente utiliza de forma habitual método anticonceptivo
+ c_num: Número de consultas
+ motiles: Motivo por el cual se desea la interrupción del embarazo
+ h_ingreso: En caso de hospitalización, fecha de ingreso
+ desc_servicio: Servicio en el que se otorgó la interrupción
+ p_semgest: Semanas de gestación por USG
+ p_diasgesta: Días de gestación por USG
+ p_consent: Se requisitó y firmo el consentimiento informado
+ procile: Método utilizado para la interrupción legal
+ s_complica: Se presentaron complicaciones por el procedimiento
+ panticoncep: Método anticonceptivo proporcionado post evento obstétrico

La variable target es la edad pasada a una variable binaria que indica si es o no mayor de 23 años (>= 23 años). Queremos predecir si dadas las características de la persona que llega a realizarse un aborto es mayor o igual a 23 años o no.

Numericamente se observa que existen datos numericos y categóricos, así como algunas fechas que de inicio no se reconocen porque existen algunos errores que lo impiden

In [ ]:
#We define a maximum number of columns to display
pd.options.display.max_columns = 80
# Datos originales de entrada
#interrupcion_legal.head()

In [ ]:
########################################  LIMPIAR NOMBRE VARIABLES  ######################################################
# Estandarizacion de los nombres de las columnas
clean_interrupcion_legal=clean_data.clean_variables_names(interrupcion_legal)


In [ ]:
############################################## INFORMACION GENERAL ##################################################
# Informacion general de la base
#eda.df_variables_info(clean_interrupcion_legal)
# Clasificación de las variables por tipo
[numeric_variables, categorical_variables, dates_variables, string_variables] = eda.info_type_of_vars(clean_interrupcion_legal)
# Imprime a pantalla las variables por tipo
#eda.print_info_type_of_vars(numeric_variables, categorical_variables, dates_variables, string_variables)conf_mat.sum(1)

In [ ]:
#######################################  LIMPIAMOS VALORES DE VARIABLES  ###############################################
# Se observa que de las variables de tipo "string", 3 corresponden a variables tipo "date" que
# no fueron reconocidas, a saber: fingreso, fmenstrua, h_fingreso

# Primeramente estandarizaremos los valores del resto de las variables string
clean_interrupcion_legal = transform_data.clean_variables_values(clean_interrupcion_legal, ['mes','autoref','edocivil_descripcion','desc_derechohab','nivel_edu','ocupacion','religion','parentesco','entidad','alc_o_municipio','consejeria','anticonceptivo','motiles','desc_servicio','p_consent','procile','s_complica','panticoncep'
])
#clean_interrupcion_legal.head(30)

In [ ]:
#######################################  HACEMOS RECONOCER VARIABLES DE FECHA  ###########################################
###### Correccion de "fmenstrua" a variable tipo "date"

clean_interrupcion_legal['fmenstrua'] = pd.to_datetime(clean_interrupcion_legal['fmenstrua'])

In [ ]:
###### Correccion de "fingreso" 

#Cambiamos np.nan a NA para que sea reconocida
clean_interrupcion_legal['fingreso'] = clean_interrupcion_legal['fingreso'].replace(np.nan,"NA")
# Separamos la fingreso en 3 columnas (año, mes, dia) usando '/' como token, para identificar posibles errores
dividir = lambda x: pd.Series([i for i in x.split('/')])
aux = clean_interrupcion_legal['fingreso'].apply(dividir)
# Regresamos a los valores originales
clean_interrupcion_legal['fingreso'] = clean_interrupcion_legal['fingreso'].replace("NA", np.nan)


In [ ]:
aux = pd.DataFrame(aux)
aux.columns = ['dia','mes','ano']
# Identificamos meses > 12, para checar que no haya errores
mes_mayor_12 = aux[pd.to_numeric(aux['mes']) > 12]
#print('Observaciones con mes mayor a 12: \n', mes_mayor_12,'\n')

# Identificamos años menores a 2016 o mayores a 2019 -podrían indicar un error
ano_menor_16_mayor_19 = aux[(pd.to_numeric(aux['ano'])<16) | (pd.to_numeric(aux['ano'])>19)]
#print('Observaciones con año menor a 2016 o mayor a 2019: \n', ano_menor_16_mayor_19)

In [ ]:
# Obtenemos el indice de las observaciones con errores
no_observ = mes_mayor_12.loc[mes_mayor_12['mes']=='0719'].index.values[0]
# Corregimos los datos
clean_interrupcion_legal['fingreso'].iloc[no_observ] = '17/07/19'
# Checamos que los cambios esten en la base
#clean_interrupcion_legal['fingreso'][no_observ]

In [ ]:
#Se asjuta el año de la fecha 'fingreso' acorde a la variable 'ano' (se asume que el año difiere por ser un error)
temp = ano_menor_16_mayor_19.index.values
for i in range(0,len(temp)):
    # Corregimos el dato
    clean_interrupcion_legal['fingreso'].iloc[temp[i]] = ano_menor_16_mayor_19.iloc[i,0] + '/' + ano_menor_16_mayor_19.iloc[i,1] + '/' + str(clean_interrupcion_legal['ano'].iloc[temp[i]])[-2:]
    #check the correction
    #print(clean_interrupcion_legal['fingreso'][temp[i]])

In [ ]:
#clean_interrupcion_legal[clean_interrupcion_legal['fingreso'].isna()==True]

In [ ]:
# Finalmente cambiamos a tipo "date" la variable "fingreso"
clean_interrupcion_legal['fingreso'] = pd.to_datetime(clean_interrupcion_legal['fingreso'])

In [ ]:
# Volmenos a clasificar las variables por tipo para actualizar los cambios
[numeric_variables, categorical_variables, dates_variables, string_variables] = eda.info_type_of_vars(clean_interrupcion_legal)
# Imprimimos las variables por tipo
# eda.print_info_type_of_vars(numeric_variables, categorical_variables, dates_variables, string_variables)

## 2.2. Limpieza de datos

Para cada variable se fueron haciendo algunas correcciones de lógica

Para la mayoría de los casos no se cuenta con información sobre el centro de salud o persona que hizo referencia al centro de salud en el que se trata a la paciente

 Existen observaciones cn 'autorefereida' y 'autorreferida' que se asume como error ortografico


 De aquí en adelante, todas los valores definidos como N/E, no especificado, ect se definirán con la misma clave 'ne'
 clean_interrupcion_legal.loc[clean_interrupcion_legal['edocivil_descripcion']=='n/e', 'edocivil_descripcion']='ne'

#Notamos que las categorías no son del todo claras, pues existen observaciones donde definen más
#de un valor como imss/issste o imss/issste/gratuidad.
#We note that the categories are not entirely clear, as there are observations where they define
#more than one value as imss/issste or imss/issste/gratuity.

#Corrección/observación entidad y alc_o_municipio...

#En la base original existe un solo registro juan c. bonilla, que debería ser juan c bonilla
#no obstante este problema quedó arreglado con la limpieza de los valores de las variables

#Corrección/observación menarca...

#Parece un error que aparezcan edades de 1,2,3,4,5,6,... años como edad de primera
#menstruación. Estos valores se repiten varias veces aunque no dejan de ser poco representativos

#Se podría considerar cambiar los valores de 1-6 por la mediana

Tanto en 'menarca' como en 'fsexual' y 'fmenstrua' existen valores que parecen no hacer mucho sentido, debido a que son datos con valores numericos (que representan años) muy bajos, pues la primera menstruación y/o relación sexual a los 0,1,2,3,4,5 y 6 años pareciera tratarse de un error. No obstante estas observaciones aparentemente inconsistentes son relativamente mínimas
#Se observa una fecha fmenstrua mayor a 2019, lo cual se asume como un error y se cambia por NA

Corrección/observación sememb...

#Se considera que el embarazo dura aprox. 40 semanas contadas a partir de la fecha de la
#última menstruación, sin embargo en la base de datos hay algunos valores mayores a 40 (44,44.5,49,50)

#Se puede sugerir cambiar los valores mayores a 40 por nan

Para las variables 'nhijos', 'gesta', 'naborto', 'npartos', 'ncesarea', 'nile', 'consejería' no se considera que tengan errores y sus valores faltantes son relativamente poco representativos. Cabe señalar que la diferencia entre 'naborto' y 'nile' está en que la primera indica aborto dentro o fuera de la opción legal através de los centros de saludos indicados para ello (pudiese tratarse de un microaborto, aborto clandestino, aborto en otra institución de salud, etc). 'nile' indica no. de abortos en el presente programa

h_ingreso...
#Existen datos errorneos, pues hay fechas expresadas como entero aparentemente indicando numero de días transcurridos
#Existen valores ne, para los cuales se debe tener cuidado si se pretende hacer operciones con esta variable

desc_servicio, p_semgest, p_diasgesta, p_consent, procicle, s_complica, panticoncept...
#no se encontraron anomalías con los valores de estas variables

In [ ]:
#######################################  CORRECCIÓN DE ALGUNOS ERRORES  ###########################################
# Corrección/observación autoref...

# Existen observaciones con 'autorefereida' y 'autorreferida' que se asume como error ortografico
clean_interrupcion_legal.loc[clean_interrupcion_legal['autoref']=='autoreferida', 'autoref']='autorreferida'

In [ ]:
#Corrección/observación desc_derechohab...

#clean_interrupcion_legal.loc[clean_interrupcion_legal['edocivil_descripcion']=='n/e', 'edocivil_descripcion']='ne'

In [ ]:
#Corrección/observación fsexual y fmenstrua...

#Se observa una fecha fmenstrua mayor a 2019, lo cual se asume como un error y se cambia por NA
clean_interrupcion_legal.loc[clean_interrupcion_legal['fmenstrua'] > pd.Timestamp(date(2019,12,31)), 'fmenstrua']=pd.to_datetime(np.nan)

## 2.3. Transformación de datos

Es de señalar que la variable dummy/indicadora (y variable objetivo) de 23 años o más, divide al grupo de aplicantes casi a la mitad, pues como señalamos con anterioridad la moda se ubica en 24 años

Con la finalidad de seguir haciendo hallazgos interesantes en la interacción de las diferentes variables, creamos algunas variables auxiliares, como la que categoriza los valores de 'edad'

In [ ]:
####################################### TRANSFORMACION DE DATOS #####################################################
############ Creamos la variable objetivo
#Creamos la variable objetivo
clean_interrupcion_legal['23_o_mayor'] = np.where(clean_interrupcion_legal['edad'] >= 23, 1, 0)
#clean_interrupcion_legal.head()

In [ ]:
########### Creamos la variable temporal "ano_mes_ile"
##Concatenamos las variables ano y mes (convirtiendo mes a numero)
mes_num = clean_interrupcion_legal['mes'].map(transform_data.month_string_to_number_string)
clean_interrupcion_legal['ano_mes_ile'] = clean_interrupcion_legal['ano'].astype(str) + "/" + mes_num.astype(str)
# Convertimos la variable a tipo "date"
clean_interrupcion_legal['ano_mes_ile'] = pd.to_datetime(clean_interrupcion_legal['ano_mes_ile'])

In [ ]:
########### Edad por quintiles
clean_interrupcion_legal['edad_quintiles'] = pd.cut(clean_interrupcion_legal['edad'], bins=[0,4,9,14,19,24,29,34,39,44,49,54,59,100], labels=["0 a 4", "5 a 9", "10 a 14", "15 a 19", "20 a 24", "25 a 29", "30 a 34", "35 a 39", "40 a 44", "45 a 49", "50 a 54", "55 a 59", "60 y mas"], right=True)

In [ ]:
########### Recategorizacion de *estado_civil*
dic_edo = {'soltera':'soltera', 'divorciada':'divorciada', 'union_libre':'union_libre', 'casada':'casada', 'separada':'separada', 'n/e': 'no_especificado', 'na':'no_especificado'}
clean_interrupcion_legal['edo_civil'] = clean_interrupcion_legal['edocivil_descripcion'].map(dic_edo)

In [ ]:
########### Recategorizacion de *nivel_educativo*
dic_escol = { 'ninguno':'ninguno', 'preescolar':'ninguno', 'primaria_completa':'primaria', 'primaria_incompleta':'primaria',
             'secundaria_incompleta':'secundaria', 'secundaria_completa':'secundaria', 'preparatoria_incompleta':'media_superior',
             'preparatoria_completa':'media_superior', 'carrera_tecnica':'superior', 'licenciatura_incompleta':'superior','licenciatura_completa':'superior',
             'maestria':'posgrado', 'doctorado':'posgrado', 'posgrado':'posgrado', 'posgrado_incompleto':'posgrado',
             'no_especifica':'no_especificado', 'NA':'no_especificado', 'otra':'otra'}

clean_interrupcion_legal['escolaridad'] = clean_interrupcion_legal['nivel_edu'].map(dic_escol)

In [ ]:
########### Recategorizacion de *ocupacion*
dic_ocup = {'abogada':'profesionista', 'administradora':'profesionista', 'ama_de_casa':'ama_de_casa', 'arquitecta':'profesionista',
            'artista':'profesionista', 'asesora_financiera':'profesionista', 'asesora_juridica':'profesionista', 'auditora':'profesionista',
            'bibliotecaria':'profesionista', 'cajera':'empleada', 'capturista':'empleada', 'chofer':'empleada', 'cientifica':'profesionista',
            'cocinera':'empleada', 'comerciante':'empleada', 'constructora_o_elecetricista':'empleada', 'constructura_o_elecetricista':'empleada',
            'contadora':'empleada', 'dentista':'profesionista', 'desempleada':'desempleada', 'diseñadora':'profesionista',
            'ejecutiva':'profesionista', 'empleada':'empleada', 'enfermera':'profesionista', 'estilista':'empleada',
            'estudiante':'estudiante', 'fisioterapeuta':'empleada', 'fotografa':'empleada', 'informatica_o_tecnologia':'profesionista',
            'ingeniera':'profesionista', 'logistica_o_eventos':'empleada', 'mecanica':'profesionista', 'medico':'profesionista',
            'mesera':'empleada', 'modelo':'empleada', 'na':'no_especificado', 'nutiologa':'profesionista', 'obrera':'empleada',
            'paramedico':'empleada', 'periodista_o_publicista':'profesionista', 'policia_o_seguridad':'empleada',
            'profesora_o_educadora':'profesionista', 'psicologa':'profesionista', 'recepcionista':'empleada', 'recursos_humanos':'empleada',
            'secretaria':'empleada', 'telefonista':'empleada', 'textil':'empleada', 'trabajadora_de_campo':'empleada',
            'trabajadora_del_hogar':'empleada', 'trabajadora_del_sector_publico':'empleada', 'trabajadora_sexual':'trabajadora_sexual',
            'vetarinaria':'profesionista', 'voluntaria_o_trabajadora social':'empleada'}
clean_interrupcion_legal['ocupacion2'] = clean_interrupcion_legal['ocupacion'].map(dic_ocup)

In [ ]:
########### Recategorizacion de *anticonceptivo*
dic_anti = { 'ninguno':'ninguno', 'na':'ninguno','condon':'condon', 'condon_y_otro':'otro',
            'anillo_vaginal':'otro', 'anticoncepcion_de_emergencia':'otro', 'anticoncepcion_de_emergencia_y_otro':'otro',
            'barrera':'otro', 'calendario':'otro', 'coito_interrumpido':'otro', 'diu':'otro', 'hormonal_inyectable':'otro',
            'hormonal_oral':'otro', 'implante_subdermico':'otro', 'inyeccion':'otro', 'medicamento':'otro',
            'parche_dermico':'otro', 'pastillas_anticonceptivas':'otro', 'ritmo':'otro', 'salpingoclacia':'otro',
            'vasectomia':'otro'}
clean_interrupcion_legal['anticonceptivo2'] = clean_interrupcion_legal['anticonceptivo'].map(dic_anti)

In [ ]:
# Actualizamos las variables por tipo
[numeric_variables, categorical_variables, dates_variables, string_variables] = eda.info_type_of_vars(clean_interrupcion_legal)
# Imprime las variables por tipo
#eda.print_info_type_of_vars(numeric_variables, categorical_variables, dates_variables, string_variables)

# 3. Data profiling

A continuación se muestra el dataprofiling de nuestras variables resultantes para una fácil identificación de estadísticos (mismos que en forma general ya fueron comentados arriba). 
Tambien se recomienda consultar el pandas profiling que este código genera

In [ ]:
# Data profiling para variables numéricas
eda.descriptive_stats_for_numeric_vars(clean_interrupcion_legal, numeric_variables)

La siguiente table indica las estadísticas de la variable categóricas 

In [ ]:
# Data profiling para variables categoricas
eda.descriptive_stats_for_categorical_vars(clean_interrupcion_legal, string_variables)

In [ ]:
clean_interrupcion_legal.shape

# 4. EDA

La mayoría son mujeres solteras o en unión libre
La mayoría de las aplicantes son jovenes de entre 20 y 30 años, inclusive el promedioo y moda son similares (25.7 y 24)
'desc_derechohab' indica a qué servicio de salud está afiliada la paciente, no obstante hay valores que indican multiples opciones, lo cual hace un tanto confusa y quizá imprecisa esta variable
Más del 70% de las mujeres tiene un nivel educativo inferior a estudios universitarios, mismas que en su mayoría tienen concluida la secundaria o prepa
Un 40% de las mujeres dependen de una fuente externa de ingresos (amas de casa o estudiantes)
La mayoría de las personas son de religión católica o dicen no tener alguna creencia en particular (lo cual suena congruente con la población del país)
Esta variable tiene muchos faltantes, se podría asumir que es debido a que no tuvieron algún acompañante, porque sólo es obligatorio para menores de edad acorde a la pagina web. Entre los acompañantes más solicitados está la madre, la pareja y el esposo. No obstante lo anterior, se tiene que tomar en cuenta que para cualquier procedimiento médico que se aplique se pide un acompañante, por lo que los valores faltantes cobran importancia y restan importancia predictiva a la variable
El 95% de las personas son de la zona metropolitana (principalmente de la CDMX). En cuanto a las alcaldías/municipio llama la atención que un 19% son o de la alcaldía Iztapalapa o Gustavo A Madero

montiles' es una variable que indica los motivos por los que se realiza la interrupción del embarazo, lo cual sugiere que guarda un buen nivel predictivo para con nuestra variable objetivo, sin embargo la mayoría (97.4%) de los valores indican "interrupción voluntaria", lo que indica que no se reportó un motivo concreto útil

Las variables asociadas al evento posintervención no fueron consideradas para el análsis, pues se considera que no aportan información útil además de que podrían sugerir data leaking


EXPLICAR POR QUE SE HACE ANO-MES O USAR FINGRESO

Para efectos de separar la base en "entrenamiento" y "prueba", se considerará la temporalidad de los datos, esto porque variables cmo 'nile' sugiere que podrían existir personas que reincidan en la practica de interrupción legal del embarazo

Los estadísticos de 'edad_quintiles' dejan claro que la mayoría de las mujeres son jóvenes de entre 20 y 29 años


In [ ]:
sns.set_style("darkgrid")
fig, ax = plt.subplots(figsize=(20,10))
clean_interrupcion_legal.groupby(['ano_mes_ile']).count()['ano'].plot(ax=ax,color='darkblue', marker='o')

plt.title('ILE por mes \n (2016-2019)', fontsize='16')
plt.xticks(fontsize='14')
plt.yticks(fontsize='14')
plt.show()

In [ ]:
sns.set_style("darkgrid")

fig, ax = plt.subplots(figsize=(15,7))

g = clean_interrupcion_legal.groupby(['ano_mes_ile', '23_o_mayor']).count()['mes'].unstack().plot(ax=ax, marker='o', color= ['gray', 'blue'])

ax.xaxis.grid(True, which='minor')

leg = g.axes.get_legend()
leg.set_title('')
new_labels=['Menores a 23 años', '23 años o más']
for t, l in zip(leg.texts, new_labels): t.set_text(l)


plt.ylabel("")
plt.xlabel("")
plt.title("Número de ILE por mes y rango de edad\n (2016-2019)")

plt.show()

In [ ]:
clean_interrupcion_legal.groupby(['ocupacion2', '23_o_mayor'])['mes'].count()

In [ ]:
sns.set_style("darkgrid")
plt.figure(figsize = (20,10))
#plt.style.use('ggplot')
plot1 = sns.boxplot(y='edad', x='nhijos', data=clean_interrupcion_legal, palette='dark')
sns.stripplot(y='edad', x='nile', data=clean_interrupcion_legal, color="gray", jitter=0.2, size=2.5)
# Linea horizontal
plot1.axes.axhline(y = 23, ls='--', color='red')
plot1.axes.text(-0.8, 22.7, "23 años", color = 'red', fontsize=12)

plt.title('Distribución de la edad por número de ILE', fontsize='16')
plt.xlabel('Número de Interrupciones Legales (ILE)', fontsize='14')
plt.ylabel('Edad de la mujer', fontsize='14')

plt.show()

In [ ]:
tabla_pivote = pd.pivot_table(clean_interrupcion_legal, index='ocupacion2',  aggfunc='count')
#tabla_pivote = tabla_pivote.reindex(['casada', 'union_libre', 'soltera', 'divorciada', 'separada','no_especificado'])
tabla_pivote['edad']

In [ ]:
ile_menor_23 = tabla_pivote['edad'][0]/sum(tabla_pivote['edad'][0])*100
ile_mayor_23 = tabla_pivote['edad'][1]/sum(tabla_pivote['edad'][1])*100

In [ ]:
df_aux = pd.DataFrame(tabla_pivote['edad'][0]/sum(tabla_pivote['edad'][0])*100)
df_aux = pd.concat([df_aux, tabla_pivote['edad'][1]/sum(tabla_pivote['edad'][1])*100])
df_aux['23_0_mas'] = [0,0,0,0,0,0,1,1,1,1,1,1]
df_aux['edo_civil1'] = ['casada', 'union_libre', 'soltera', 'divorciada', 'separada','no_especificado', 'casada', 'union_libre', 'soltera', 'divorciada', 'separada','no_especificado']
df_aux.columns = ['Porcentaje', '23_o_mayor', 'edo_civil1']
df_aux.index = df_aux.reset_index(level=0, drop=True)
df_aux.index = [0,1,2,3,4,5,6,7,8,9,10,11]
g = sns.catplot(y="Porcentaje", x='edo_civil1', hue='23_o_mayor', kind="bar", data=df_aux, palette=['gray', 'blue'], height=6, aspect=1.3, legend_out=False)
g.ax.set_yticks(np.arange(0,110,10), minor=True)

leg = g.axes.flat[0].get_legend()
leg.set_title('')
new_labels=['Menores a 23 años', '23 años o más']
for t, l in zip(leg.texts, new_labels): t.set_text(l)
    
plt.ylabel("")
plt.xlabel("")
plt.title("Porcentaje de mujeres que recibieron una ILE por Estado Civil")


plt.show()


In [ ]:
tabla_pivote = pd.pivot_table(clean_interrupcion_legal, index='escolaridad', columns='23_o_mayor', aggfunc='count')
tabla_pivote = tabla_pivote.reindex(['ninguno', 'primaria', 'secundaria', 'media_superior', 'superior','posgrado', 'otra', 'no_especificado'])
tabla_pivote['edad']

df_aux = pd.DataFrame(tabla_pivote['edad'][0]/sum(tabla_pivote['edad'][0])*100)
df_aux = pd.concat([df_aux, tabla_pivote['edad'][1]/sum(tabla_pivote['edad'][1])*100])
df_aux['23_0_mas'] = [0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1]
df_aux.index = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]
df_aux['escolaridad1'] = ['ninguno', 'primaria', 'secundaria', 'media \n superior', 'superior','posgrado', 'otra', 'no_especificado', 'ninguno', 'primaria', 'secundaria', 'media \n superior', 'superior','posgrado', 'otra', 'no_especificado']
df_aux.columns = ['Porcentaje', '23_o_mayor', 'escolaridad1']
df_aux
#
g = sns.catplot(y="Porcentaje", x='escolaridad1', hue='23_o_mayor', kind="bar", data=df_aux, palette=['gray', 'blue'], height=6, aspect=1.3, legend_out=False)
g.ax.set_yticks(np.arange(0,110,17), minor=True)

leg = g.axes.flat[0].get_legend()
leg.set_title('')
new_labels=['Menores a 23 años', '23 años o más']
for t, l in zip(leg.texts, new_labels): t.set_text(l)
    
plt.ylabel("")
plt.xlabel("")
plt.title("Porcentaje de mujeres que recibieron una ILE por Escolaridad")
plt.show()



En la presente gráfica se observan diferencias entre las mujeres de 23 años o más y las de menos, acorde a estado civil, lo cual sugiere que el edo civil tiene 

In [ ]:
df_aux = clean_interrupcion_legal[clean_interrupcion_legal.edo_civil.isin(['casada', 'soltera','union_libre'])][['edo_civil', 'nile', 'nhijos', '23_o_mayor']]
pd.pivot_table(df_aux, index=['edo_civil', 'nhijos'], columns='nile', aggfunc='count')
#sns.lmplot( y="nile", x="nhijos", data=df_aux, fit_reg=False, hue='edo_civil', legend=True)

In [ ]:
df_aux = clean_interrupcion_legal.groupby(['entidad', '23_o_mayor']).count()['mes'].unstack(level=1).reset_index().sort_values(by=1, ascending=False)
df_aux[0]= df_aux[0]/sum(df_aux[0])*100
df_aux[1]= df_aux[1]/sum(df_aux[1])*100
df_aux.columns = ['entidad', 'menor', 'mayor']

df_aux2 = df_aux[0:2]
df_aux3 = df_aux[2:]
df_aux2 = df_aux2.append(pd.Series(['otras_entidades',sum(df_aux3['menor']), sum(df_aux3['mayor'])], index=df_aux2.columns), ignore_index=True)

#labels = df_aux2['entidad']
labels = ['Ciudad \n de México', 'Estado \n de Mexico', 'Otras \n entidades']
x = np.arange(len(labels))  # the label locations
width = 0.35  # the width of the bars



fig = plt.figure(figsize=(20, 5))
gs = GridSpec(nrows=1, ncols=5)

ax1 = fig.add_subplot(gs[0,1])
ax1.bar(x - width/2, df_aux2['menor'], width, color='gray', label='Menores de 23 años')
ax1.bar(x + width/2, df_aux2['mayor'], width, color='blue', label='23 años o más')
ax1.set_xticks(x)
ax1.set_xticklabels(labels)
ax1.set_yticks(np.arange(0,110,10), minor=True)
ax1.set_title("Top de Entidad de Residencia")
ax1.legend(loc='upper right')

ax2 = fig.add_subplot(gs[0,2:])
my_size=np.where(df_aux3['mayor']>df_aux3['menor'], 75, 30)
my_range = df_aux3['entidad']
ax2.vlines(x = my_range, ymin=df_aux3['menor'], ymax=df_aux3['mayor'], color='black', alpha=0.4)
ax2.axvline(x = 'queretaro', color='red', ls='--', linewidth=0.5)
ax2.scatter(my_range, df_aux3['menor'], color='gray', s=my_size, alpha=1, label='Menores de 23 años')
ax2.scatter(my_range, df_aux3['mayor'], color='blue', s=my_size, alpha=1 , label='23 años o más')
ax2.set_xticks(np.arange(len(my_range)))
ax2.set_xticklabels(my_range, rotation=90, horizontalalignment='right')
ax2.set_title("Desglose de Otras entidades federativas")
ax2.legend(loc='upper right')

fig.suptitle("Porcentaje de ILE segun la entidad donde residen")
#format_axes(fig)

plt.show()

In [ ]:
pd.options.display.float_format = '{:,}'.format
pivot_aux = pd.pivot_table(clean_interrupcion_legal, index='escolaridad', columns='edo_civil', aggfunc='count')['mes']
pivot_aux = pivot_aux.reindex(['posgrado', 'superior', 'media_superior', 'secundaria', 'primaria','otra', 'ninguno', 'no_especificado' ])
pivot_aux = pivot_aux[['casada', 'union_libre', 'soltera', 'divorciada', 'separada','no_especificado']]
pivot_aux
#sns.heatmap(pivot_aux)
#pd.pivot_table(clean_interrupcion_legal, index='escolaridad', columns=['23_o_mayor', 'edo_civil'], aggfunc='count')['mes']

In [ ]:
clean_interrupcion_legal[clean_interrupcion_legal['escolaridad']=='no_especificado']

In [ ]:
pivot_aux = pd.pivot_table(clean_interrupcion_legal, index='escolaridad', columns='ocupacion2', aggfunc='count')['mes']
pivot_aux = pivot_aux.reindex(['posgrado', 'superior', 'media_superior', 'secundaria', 'primaria','otra', 'ninguno', 'no_especificado' ])
pivot_aux 

In [ ]:
pivot_aux = pd.pivot_table(clean_interrupcion_legal, index='23_o_mayor', columns='escolaridad', aggfunc='count')['mes']
#pivot_aux = pivot_aux.reindex(['casada', 'union_libre', 'soltera', 'divorciada', 'separada','no_especificado'])
pivot_aux 

In [ ]:
####################################  PANDAS PROFILING  #########################################
# #Imprime a pantalla
# clean_interrupcion_legal.profile_report(correlations={"cramers": False})

# #Imprime a archivo

# profile = clean_interrupcion_legal.profile_report(title='Interrupcion legal del embarazo', plot={'histogram': {'bins': 8}}, correlations={"cramers": False})
# profile.to_file(output_file="Interrupcion legal del embarazo_output.html")

In [ ]:
# #guardamos la base de datos resultante a un .csv
# clean_interrupcion_legal.to_csv('clean_interrupcion_legal.csv', sep=';')

In [ ]:
#geda.boxplot_all(clean_interrupcion_legal, numeric_variables)

In [ ]:
sns.set_style("darkgrid")
sns.set_palette("dark")
geda.plot_by_pairs_grid(clean_interrupcion_legal, ['edad','naborto'])

In [ ]:
geda.plot_by_pairs_grid(clean_interrupcion_legal, ['edad','nhijos'])

In [ ]:
geda.plot_by_pairs_grid(clean_interrupcion_legal, ['edad','fsexual'])

In [ ]:
geda.plot_bivariate_hist(clean_interrupcion_legal, 'edad', 'naborto')

In [ ]:
geda.barplot_by_category(clean_interrupcion_legal, 'edad', 'ocupacion2')

In [ ]:
geda.barplot_by_category(clean_interrupcion_legal, 'edad', 'ocupacion2')

In [ ]:
geda.barplot_by_category(clean_interrupcion_legal, 'edad', 'edocivil_descripcion')


# Insights más importantes de este conjunto de datos

+ La mayoría de mujeres poseen una educación máxima de media superior o secundaria, con estudios terminados o incompletos. La educación se podría considerar como un factor relevante en la toma de decisiones de las personas, que abarca la planeación a futuro

+ Las mayoría de mujeres indicó ser solteras y de estas la mayoría son jóvenes. El hecho de tener algún compromiso para con una pareja o hijos pareciera otorgar mayor libertad para tomar una decisión sobre la interrupción del embarazo.

+ Considerando que la base contiene la información de diversos años, y que existen mujeres que se han practicado más de un aborto, podría indicar que hay mujeres que hacen uso de la interrupción legal de embarazo varias veces. Esto podría sesgar las predicciones dado que podríamos tener el caso de registrar a una misma persona varias veces

+ Las semanas/dias de gestación guardan una aparente fuerte correlación lineal positiva con la edad. Esto podría ser un indicativo del tiempo que se dan las personas para una toma de decisión, donde las personas más jovenes tienen a tomar la decisión en forma más temprana

+ Dado que el objetivo planteado es predecir la edad de las mujeres dadas sus características, se han descartado algunas variables como las relacionadas a eventos posteriores a la intervención, cuya consideración podría incidir en hacer data leaking. Existen otras variables que aportan mucha información al prevalecer una categoría o valor como altamente representativo, como puede ser la religión. Existen variables que a primera vista podrían sugerir un alto nivel predictivo, como son motiles (motivos por los cuales se realiza la intervención) o parentesco (persona que acompaña a la paciente), sin embargo se detecta que las respuestas a estos concepto no son de mucha ayuda. En motile la mayoría indica tal cual interrupción voluntaria, lo cual no aporta nada). Aunque se sabe que para cualquier intervención se pide se vaya acompañada, la mayoría de las personas no reporta información sobre el parentesco del acompañante

# 5. Selección de variables

Algunas variables categoricas se recategorizaron para agrupar etiquetas similares y disminuir el número de categorias.

Originalmente se tienen 36 variables, y aunque no es un gran número de variables en términos de los modelos de aprendizaje de máquina, es importante hacer la selección de variables para eliminar aquellas ue son espurias y que pudieran estar incorporando ruido al modelo y por lo tanto, disminuir su desempeño.

En base al EDA se incluyeron 10 variables (edo_civil, escolaridad, ocupacion2, menarca, anticonceptivo2, nhijos, naborto, npartos, ncesarea, nile) para correr un modelo de Random Forest para determinar el nivel de importancia de las variables.

Referencia [aqui](https://www.datacamp.com/community/tutorials/random-forests-classifier-python)


## 5.1. Separacion de datos de *entrenamiento* y *prueba*

La partición de la base en el conjunto de entrenamiento y de prueba, se realizo usando temporal cross validation, pues los eventos considerados (ILE) están intrínsicamente relacionados con el tiempo. Además, las características de la observaciones en la base, y de la población objetivo -mujeres de 23 años o más- en particular, son cambiantes a lo largo del tiempo.

La muestra de entrenamiento se selecciono desde la fecha más antigua del primer registro (2016-01-01) hasta la fecha límite elegida, en este caso 2018-07-01. Así, la muestra de prueba fueron las observaciones a partir de 2018-08-01 y hasta la última fecha disponible en la base (2019-07-01).

Esta fecha se eligio para tener al menos un año en la muestra de prueba, pensando que puede haber estacionalidad a lo largo de un año. Además, con esta fecha se obtiene que el 74% de los datos se encuentran en la muestra de entrenamiento y el restante en la muestra de prueba.

In [ ]:
####################################### SELECCION DE VARIABLES SEGUN EDA #############################################
base_interrupcion_legal = clean_interrupcion_legal[['ano_mes_ile', '23_o_mayor', 'edo_civil', 'escolaridad', 'ocupacion2', 'menarca', 'anticonceptivo2', 'nhijos', 'naborto', 'npartos', 'ncesarea', 'nile' ]].copy()

#Ordenamos los datos conforme a la variable temporal
base_interrupcion_legal.sort_values(by=['ano_mes_ile'], ascending=True)

#base_interrupcion_legal

In [ ]:
##################################### SEPARACION DE LA BASE EN "PRUEBA" Y "ENTRENAMIENTO" ###############################
# Lo deje hasta julio de 2018 para tener un anio de prueba
FECHA_DE_CORTE = '2018-07-01'
mask = (base_interrupcion_legal['ano_mes_ile'] <= FECHA_DE_CORTE)
entrenamiento = base_interrupcion_legal[mask]
prueba = base_interrupcion_legal[~mask]
print('No. observaciones en entre: ', entrenamiento.shape[0], ' que representa el ', round(entrenamiento.shape[0]/base_interrupcion_legal.shape[0]*100,0),'%')
print('No. observaciones en prueb: ', prueba.shape[0], ' que representa el ', round(prueba.shape[0]/base_interrupcion_legal.shape[0]*100,0),'%')

In [ ]:
y_entrenamiento = entrenamiento['23_o_mayor']
x_entrenamiento = pd.DataFrame(entrenamiento.drop(['23_o_mayor', 'ano_mes_ile'], axis = 1))

y_prueba = prueba['23_o_mayor']
x_prueba = pd.DataFrame(prueba.drop(['23_o_mayor', 'ano_mes_ile'], axis = 1))

In [ ]:
#eda.descriptive_stats_for_numeric_vars(x_entrenamiento, numeric_variables)
#eda.descriptive_stats_for_categorical_vars(x_entrenamiento, string_variables)
#x_entrenamiento['nile'].value_counts()
#base_interrupcion_legal["naborto"].value_counts()
#x_entrenamiento.shape

## 5.2. Imputacion de variables

Se realizaron imputaciones a las siguientes variables de la muestra de entrenamiento:

| Variable | % missings | Valor de imputación | Justificacion |
|----------|------------|---------------------|---------------|
| edo_civil | 0.32 | moda (soltera) | 55% de los datos corresponden a esa categoría |
|escolaridad | 2.54 | moda (media_superior) | 45% de los datos corresponden a esa categoría |
|menarca| 5.05 | moda (12 años) | %27 de los datos corresponden a esa categoría |
|anticonceptivo|12.50|constante (ninguno)| 53% de los datos corresponden a esa categoría|
|ocupacion| 15.91 | constante (no_especificado)| Nueva categoria para aislar el efecto de los valores faltantes|
|nhijos| 5.32 | mediana | 40% de los datos corresponden a esa categoría y la media y la mediana de los datos es muy similar|
|naborto| 6.47 | mediana |80% de los datos corresponden a esa categoría y la media y la mediana de los datos es muy similar|
|nparto|6.16| mediana | 56% de los datos corresponden a esa categoría y la media y la mediana de los datos es muy similar|
|ncesarrea|6.64| mediana | 70% de los datos corresponden a esa categoría y la media y la mediana de los datos es muy similar|
|nile|26.4| mediana | 63% de los datos corresponden a esa categoría y la media y la mediana de los datos es muy similar|

>> Porcentajes respecto a la muestra de entrenamiento

In [ ]:
################################# IMPUTACION DE VARIABLES PARA "ENTRENAMIENTO" #########################################

df = x_entrenamiento

var = 'edo_civil'
aux = df[[var]]
aux_imputer_edo_civil = SimpleImputer(missing_values=np.nan, strategy="most_frequent")
aux_imputer_edo_civil.fit(aux)
aux_imputed_edo_civil = aux_imputer_edo_civil.transform(aux)
df[var] = aux_imputed_edo_civil

var = 'escolaridad'
aux = df[[var]]
aux_imputer_escolaridad = SimpleImputer(missing_values=np.nan, strategy="most_frequent")
aux_imputer_escolaridad.fit(aux)
aux_imputed_escolaridad = aux_imputer_escolaridad.transform(aux)
df[var] = aux_imputed_escolaridad

var = 'menarca'
aux = df[[var]]
aux_imputer_menarca = SimpleImputer(missing_values=np.nan, strategy="most_frequent")
aux_imputer_menarca.fit(aux)
aux_imputed_menarca = aux_imputer_menarca.transform(aux)
df[var] = aux_imputed_menarca


var = 'anticonceptivo2'
aux = df[[var]]
aux_imputer_anticonceptivo2 = SimpleImputer(missing_values=np.nan, strategy="constant", fill_value="ninguno")
aux_imputer_anticonceptivo2.fit(aux)
aux_imputed_anticonceptivo2 = aux_imputer_anticonceptivo2.transform(aux)
df[var]=aux_imputed_anticonceptivo2


var = 'ocupacion2'
aux = df[[var]]
aux_imputer_ocupacion2 = SimpleImputer(missing_values=np.nan, strategy="constant", fill_value="no_especificado")
aux_imputer_ocupacion2.fit(aux)
aux_imputed_ocupacion2 = aux_imputer_ocupacion2.transform(aux)
df[var]=aux_imputed_ocupacion2


var = 'nhijos'
aux = df[[var]]
aux_imputer_nhijos = SimpleImputer(missing_values=np.nan, strategy="median")
aux_imputer_nhijos.fit(aux)
aux_imputed_nhijos = aux_imputer_nhijos.transform(aux)
df[var]=aux_imputed_nhijos

var = 'naborto'
aux = df[[var]]
aux_imputer_naborto = SimpleImputer(missing_values=np.nan, strategy="median")
aux_imputer_naborto.fit(aux)
aux_imputed_naborto = aux_imputer_naborto.transform(aux)
df[var]=aux_imputed_naborto

var = 'npartos'
aux = df[[var]]
aux_imputer_nparto = SimpleImputer(missing_values=np.nan, strategy="median")
aux_imputer_nparto.fit(aux)
aux_imputed_nparto = aux_imputer_nparto.transform(aux)
df[var]=aux_imputed_nparto

var = 'ncesarea'
aux = df[[var]]
aux_imputer_ncesarea = SimpleImputer(missing_values=np.nan, strategy="median")
aux_imputer_ncesarea.fit(aux)
aux_imputed_ncesarea = aux_imputer_ncesarea.transform(aux)
df[var]=aux_imputed_ncesarea

var = 'nile'
aux = df[[var]]
aux_imputer_nile = SimpleImputer(missing_values=np.nan, strategy="median")
aux_imputer_nile.fit(aux)
aux_imputed_nile = aux_imputer_nile.transform(aux)
df[var]=aux_imputed_nile



In [ ]:
####### Checamos que, en efecto, no haya missings
[numeric_variables, categorical_variables, dates_variables, string_variables] = eda.info_type_of_vars(x_entrenamiento)
eda.descriptive_stats_for_numeric_vars(x_entrenamiento, numeric_variables)
eda.descriptive_stats_for_categorical_vars(x_entrenamiento, string_variables)

In [ ]:
######## One-hot encoder Para Entrenamiento
dummies = feature_selection.transforma_cat_dummies(x_entrenamiento, string_variables, False)
x_entrenamiento = pd.concat([x_entrenamiento, dummies], axis = 1)
x_entrenamiento = x_entrenamiento.drop(string_variables, axis = 1)

In [ ]:
#x_entrenamiento.head()

## 5.3. Random forest  para selección de variables

El Random Forest se corrio con 100, 1,000 y 10,000 árboles y se determino un threshold para la importancia, de 0.04

. Se observo que,todos los casos tuvieron el mismo orden de relevancia para las variables.

Nos quedamos con las siguientes variables:

    - menarca
    - nhijos
    - ocupacion2_estudiante
    - npartos
    - escolaridad_superior
    - ocupacion2_empleada
    - ncesarea


In [ ]:
############################################# FEATURE ENGINEERING ####################################################

N_ARBOLES = 10000  #The number of trees in the forest
SEMILLA = 104 # the seed used by the random number generator

#Create a Gaussian Classifier
clf = RandomForestClassifier(n_estimators = N_ARBOLES, random_state = SEMILLA, n_jobs=-1)

#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(x_entrenamiento, y_entrenamiento)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

feature_imp = pd.Series(clf.feature_importances_, index=x_entrenamiento.columns).sort_values(ascending=False)

In [ ]:
sns.set_style("darkgrid")
pp = sns.cubehelix_palette(30, start=.1, rot=-.75, reverse=True)
fig = plt.figure(figsize=(8, 12))
#Visualizing Important Features
g = sns.barplot(x=feature_imp, y=feature_imp.index, palette=pp)
# Add labels to your graph
g.axes.axvline(x = 0.04, ls='--', color='red')
plt.xlabel('Score')
plt.ylabel('Variables')
plt.title("Vizualizacion del score de importancia de las variables")
plt.show()


In [ ]:
#################### SELECCION DE VARIABLES PARA LA MUESTRA DE ENTRENAMIENTO #########################################
x_entrenamiento = x_entrenamiento[['menarca', 'nhijos', 'ocupacion2_estudiante', 'npartos', 'escolaridad_superior', 'ocupacion2_empleada', 'ncesarea']]

# Salvamos la muestra de prueba
x_entrenamiento.to_csv("Data/x_entrenamiento.csv", index=False)

# 6. Seleccion del Modelo 

El ajuste de los parámetros se hizo para tres modelos distintos: Decision Tree, Random Forest y XGBoost. 

Para cada conjunto de parámetros, el modelo uso *temporal cross validation* con *10 folds*.

### Justificación de *temporal cross-validation*

Se uso temporal cross-validation pues el evento, en este caso el aborto, sucede en el tiempo.

### Métrica usada 

Dado que consideramos que el uso de este modelo podria ser usado para promover politicas publicas de salud, asumimos que los recursos son finitos y por tanto se busca optimizar los mismos reduciendo al maximo los falsos positivos

Precison at k:
Podemos considerar que se tienen recursos finitos, por ejemplo un presupuesto limite, por lo tanto solo se puede contratar k medicos para realizar los procedimientos o k trabajadoras sociales en educacion sexual. En el caso de se trabaje con voluntarios que no estan sujetos a una restriccion presupuestaria, podriamos tomar a k como los consultorios fisicos en los que se puede trabajar.

## 6.1. Hiperparámetros del modelo (Magic Loop)

Cada modelo se corrio considerando los siguientes rangos para los parámetros:}


|Modelo| Parametros|# de modelos| Tiempo de ejecución (mins)| Mejor score |
|:------|:-----------|------------|---------------------------|-------------|
|Decision Tree| max_depth: [1,5,10,20,50,100,500,1000], |  3,200 |  1 | 0.82 |
|             | min_samples_split: [2,5,10,20,50], | | | |
|             | criterion:['gini', 'entropy'], | | | |
|             | splitter:['random', 'best'], | | | |
|             | presort:[False, True] | | | |
|Random Forest| n_estimators: [100, 500, 1000, 5000], | 2,000| 83 | 0.83|
|             | max_depth: [5,10,20,50,100], | | | |
|             | max_features: ['sqrt','log2'], | | | |
|             | min_samples_split: [2,5,10,20,50] | | | |
|XGBoost| max_depth: [5,10,20,50,100], | 2,250 | 20| 0.83 |
|       | *min_child_weight*: [2,5,10], | | | |
|       | *learning_rate* : [0.01,0.05,0.1]   | | | |         
|       | objective:['binary:logistic', 'binary:logitraw', 'binary:hinge'] | | | |


'max_depth': [5,10,20,50,100],
                    'n_estimators': [2,5,10,20,50],
                    'min_samples_split': [2,5,10,20,50]
                    'learning_rate':[0.01,0.05,0.1],
                    'loss':['deviance', 'exponential]

Probando 3 modelos diferentes con distinto hiperparametros, donde seleccionamos el mejor modelo con base en el score F1, que relaciona las metricas *precision* y *recall*.

Lo anterior considerando que el uso de estos resultados podrá usarse para promover alguna politica pública de salud, donde los recursos son finitos e interesa seleccionar con precison a las personas objetivo.

**Los modelos se corrieron en una Mac Pro con 23 MB en memoria y 8 cores.**

A continuación se muestran los resultados de la métrica seleccionada para cada modelo.

Referencia [aqui](https://medium.com/@dcamarena0229/time-series-split-with-scikit-learn-de7ec17d69cd)

In [ ]:
########################################## TEMPORAL CROSS VALIDATION #################################################
tscv = TimeSeriesSplit(n_splits=10)

print(tscv)

X = x_entrenamiento
y = y_entrenamiento

cv = [(train_index, test_index)
      for train_index, test_index in tscv.split(X)]
#    print("TRAIN:", train_index, "TEST:", test_index)
#    X_train, X_test = X.iloc[train_index, :], X.iloc[test_index,:]
#    y_train, y_test = y.iloc[train_index], y.iloc[test_index]   

In [ ]:
############################################### DECISION TREE ########################################################
#ocuparemos un RF
classifier = DecisionTreeClassifier()

##separando en train, test
#X_train, X_test, y_train, y_test = train_test_split(X, y)

#definicion de los hiperparametros que queremos probar
hyper_param_grid1 = {'max_depth': [1,5,10,20,50,100,500,1000], 
                     'min_samples_split': [2,5,10,20,50],
                    'criterion':['gini', 'entropy'],
                    'splitter':['random', 'best'],
                    'presort':[False, True]}

#ocupemos grid search!, el verbose permite hacer debugging ... el 3 nos permitirá ver los mensajes de 3 trials de los 10 
grid_search_1 = GridSearchCV(classifier, 
                           hyper_param_grid1, 
                           scoring = 'f1',
                           cv = cv, 
                           n_jobs = -1,
                           verbose = 3)
grid_search_1.fit(x_entrenamiento, y_entrenamiento)

#de los valores posibles que pusimos en el grid, cuáles fueron los mejores
grid_search_1.best_params_

#mejor score asociado a los modelos generados con los diferentes hiperparametros
#corresponde al promedio de los scores generados con los cv
print('Valor de la mejor métrica para el modelo de Decision Tree:', grid_search_1.best_score_ * 100)

In [ ]:
cv_results_arbol = pd.DataFrame(grid_search_1.cv_results_)
cv_results_arbol.head()
#
cv_results_arbol.to_csv("Output/DecisionTree.csv", index=False)

In [ ]:
############################################### RANDOM FOREST ########################################################
#ocuparemos un RF
classifier = RandomForestClassifier()

##separando en train, test
#X_train, X_test, y_train, y_test = train_test_split(X, y)

#definicion de los hiperparametros que queremos probar
hyper_param_grid2 = {'n_estimators': [100, 500, 1000, 5000], 
                    'max_depth': [5,10,20,50,100], 
                    'max_features': ['sqrt','log2'],
                    'min_samples_split': [2,5,10,20,50]}

#ocupemos grid search!, el verbose permite hacer debugging ... el 3 nos permitirá ver los mensajes de 3 trials de los 10 
grid_search_2 = GridSearchCV(classifier, 
                           hyper_param_grid2, 
                           scoring = 'f1',
                           cv = cv, 
                           n_jobs = -1,
                           verbose = 3)
grid_search_2.fit(x_entrenamiento, y_entrenamiento)

#de los valores posibles que pusimos en el grid, cuáles fueron los mejores
grid_search_2.best_params_

#mejor score asociado a los modelos generados con los diferentes hiperparametros
#corresponde al promedio de los scores generados con los cv
print('Valor de la mejor métrica para el modelo de Random Forest:', grid_search_2.best_score_ * 100)

In [ ]:
cv_results_forest = pd.DataFrame(grid_search_2.cv_results_)
cv_results_forest.head()
#Save to file
cv_results_forest.to_csv("Output/RandomForest.csv", index=False)

In [ ]:
############################################### XGBOOST ########################################################
#ocuparemos un RF
classifier = GradientBoostingClassifier()
#classifier = xgb.XGBClassifier()

##separando en train, test
#X_train, X_test, y_train, y_test = train_test_split(X, y)

#definicion de los hiperparametros que queremos probar
hyper_param_grid3 = {'max_depth': [5,10,20,50,100],
                    'n_estimators': [2,5,10,20,50],
                    'min_samples_split': [2,5,10,20,50],
                    'learning_rate':[0.01,0.05,0.1],
                    'loss':['deviance', 'exponential']}

#ocupemos grid search!, el verbose permite hacer debugging ... el 3 nos permitirá ver los mensajes de 3 trials de los 10 
grid_search_3 = GridSearchCV(classifier, 
                           hyper_param_grid3, 
                           scoring = 'f1',
                           cv = cv, 
                           n_jobs = -1,
                           verbose = 3)
grid_search_3.fit(x_entrenamiento, y_entrenamiento)

#de los valores posibles que pusimos en el grid, cuáles fueron los mejores
grid_search_3.best_params_

#mejor score asociado a los modelos generados con los diferentes hiperparametros
#corresponde al promedio de los scores generados con los cv
print('Valor de la mejor métrica para el modelo de XGBoost:', grid_search_3.best_score_ * 100)

In [ ]:
cv_results_XGBoost = pd.DataFrame(grid_search_3.cv_results_)
cv_results_XGBoost.head()
#
cv_results_XGBoost.to_csv("Output/XGBoost.csv", index=False)

## 6.2. Selección del modelo y parámetros
Comparando los resultados de todos los modelos considerados, tenemos que el modelo seleccionado debería ser un *XGBoost* con los siguientes parámetros

In [ ]:
########################################### SELECCION DE MODELO Y PARAMETROS #########################################
results_all = pd.concat([cv_results_forest, cv_results_arbol, cv_results_XGBoost], axis=0, sort='False').sort_values(by='rank_test_score', ascending=True).head()

results_all.to_csv("Output/all_models.csv", index=False)

## mejor configuración de hiperparámetros de acuerdo a nuestra métrica
results_all.iloc[0,:].params

In [ ]:
#cv_results_forest
#cv_results_arbol
#cv_results_XGBoost

# 7. Predicción en la muestra de prueba

A continuación se presentas las métricas con la muestra de prueba

In [ ]:
################################## PREDICCION EN LA MUESTRA DE PRUEBA ################################################
######## Imputacion para base de prueba

df = x_prueba

var = 'edo_civil'
aux = df[[var]]
aux_imputed_edo_civil = aux_imputer_edo_civil.transform(aux)
df[var] = aux_imputed_edo_civil

var = 'escolaridad'
aux = df[[var]]
aux_imputed_escolaridad = aux_imputer_escolaridad.transform(aux)
df[var] = aux_imputed_escolaridad

var = 'menarca'
aux = df[[var]]
aux_imputed_menarca = aux_imputer_menarca.transform(aux)
df[var] = aux_imputed_menarca

var = 'anticonceptivo2'
aux = df[[var]]
aux_imputed_anticonceptivo2 = aux_imputer_anticonceptivo2.transform(aux)
df[var]=aux_imputed_anticonceptivo2


var = 'ocupacion2'
aux = df[[var]]
aux_imputed_ocupacion2 = aux_imputer_ocupacion2.transform(aux)
df[var]=aux_imputed_ocupacion2

var = 'nhijos'
aux = df[[var]]
aux_imputed_nhijos = aux_imputer_nhijos.transform(aux)
df[var]=aux_imputed_nhijos

var = 'naborto'
aux = df[[var]]
aux_imputed_naborto = aux_imputer_naborto.transform(aux)
df[var]=aux_imputed_naborto

var = 'npartos'
aux = df[[var]]
aux_imputed_nparto = aux_imputer_nparto.transform(aux)
df[var]=aux_imputed_nparto

var = 'ncesarea'
aux = df[[var]]
aux_imputed_ncesarea = aux_imputer_ncesarea.transform(aux)
df[var]=aux_imputed_ncesarea

var = 'nile'
aux = df[[var]]
aux_imputed_nile = aux_imputer_nile.transform(aux)
df[var]=aux_imputed_nile



In [ ]:
######## One -hot encoder Para Prueba
dummies = feature_selection.transforma_cat_dummies(x_prueba, string_variables, False)
x_prueba = pd.concat([x_prueba, dummies], axis = 1)
x_prueba = x_prueba.drop(string_variables, axis = 1)


In [ ]:
# Seleccion de variables para la muestra de "prueba"
x_prueba = x_prueba[['menarca', 'nhijos', 'ocupacion2_estudiante', 'npartos', 'escolaridad_superior', 'ocupacion2_empleada', 'ncesarea']]

In [ ]:
#x_prueba
# Salvamos la muestra de prueba
x_prueba.to_csv("Data/x_prueba.csv", index=False)
# Salvamos y_prueba
y_prueba.to_csv("Data/y_prueba.csv", index=False, header=['label_value'] )

In [ ]:
################################################# MODELO SELECCIONADO ##############################################
# Tomado de (https://jessesw.com/XG-Boost/)

# specify parameters via map
#param_finales = {'learning_rate': 0.05, 'max_depth': 10, 'min_child_weight': 10, 'objective': 'binary:hinge'}


best_model = GradientBoostingClassifier(max_depth=5, max_features='log2', min_samples_split=2, n_estimators=100)

best_model.fit(x_entrenamiento, y_entrenamiento)

#xgdmat = xgb.DMatrix(x_entrenamiento, y_entrenamiento)
#best_model = xgb.train(param_finales, xgdmat, NUM_ROUNDS)


In [ ]:
# Salvamos el modelo a disco
filename = 'Model/final_model_ile.sav'
pickle.dump(best_model, open(filename, 'wb'))

In [ ]:
# we can then plot our feature importances using a built-in method. This is similar to the feature importances found in sklearn.
best_model.feature_importances_
#sns.set_style("darkgrid")
#xgb.plot_importance(best_model)

In [ ]:
######################################## PREDICCION PARA LA MUESTRA DE "PRUEBA" #####################################
y_predict = best_model.predict(x_prueba)
#testdmat = xgb.DMatrix(x_prueba)
#y_predict = best_model.predict(testdmat)
y_predict

In [ ]:
# Convertimos a etiquetas 0/1, SI ES NECESARIO
#THRESHOLD = 0.5

#y_predict[y_predict > THRESHOLD] = 1
#y_predict[y_predict <= THRESHOLD] = 0
#y_predict


In [ ]:
########## Métricas
print('Accuracy:', accuracy_score(y_predict, y_prueba))

In [ ]:
print('Recall:', recall_score(y_prueba, y_predict, average='weighted'))

In [ ]:
print('F1 score:', f1_score(y_prueba, y_predict, average='weighted'))

In [ ]:
########### ROC curve
fpr, tpr, thresholds = roc_curve(y_prueba, y_predict)
roc_auc = auc(fpr, tpr)

In [ ]:
sns.set_style("darkgrid")
plt.figure()
lw = 2
plt.plot(fpr, tpr, color='darkorange', lw=lw, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([-0.02, 1.0])
plt.ylim([0.0, 1.02])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC curve')
plt.legend(loc="lower right")
plt.show()

In [ ]:
########### Confusion matrix
conf_mat = pd.DataFrame(confusion_matrix(y_prueba, y_predict))
conf_mat = conf_mat[conf_mat.columns[::-1]].iloc[::-1]
print(conf_mat)
conf_mat.div(conf_mat.sum(0), axis=1).round(2)*100

Recordemos que para este proyecto $1$ indica que la edad de la mujer es de 23 años o más, mientras que $0$ indica que la edad es mejor a 23 años. Así, la matriz de confusion indica:

 - El modelo clasifica correctamente en el $76\%$ de los casos e incorrectamente en el $24\%$
 - Del total de mujeres cuya edad fue clasificada como mayor o igual a 23 años, en el $81\%$ de los casos, fueron en efecto mayores a 23 años (**Verdaderos Positivos**). Mientras que en el restante $19\%$, la predicción fue incorrecta, es decir, las mujeres tenían menos de 23 años pero fueron clasificadas como mayores o iguales a 23 (**Falsos Positivos**)
 - Del total de casos clasificados con edades menores a 23 años, el $68\%$ tenían, en efecto, menos de 23 años.  (**Verdaderos Negativos**) y el $32\%$ fueron mal clasificados pues eran mayores a 23 años (**Falsos Negativos**)

# 8. Consideraciones éticas

Es necesario considerar los siguientes aspectos del modelo que pudieran tener implicaciones éticas más allá del proyecto inicial, se presentan por etapa

  + **Planteamiento del objetivo**
    Se desconoce la finalidad del objeto de predecir si una mujer que se practica una ILE tiene 23 años o más, es decir, el uso que se le dará los analisis y modelos obtenidos. Por ejemplo, si se usa para promover políticas públicas (otorgamiento o restricción de beneficios) o para ofrecer alguna alternativa por parte del sector privado, la métrica de medición podría ser enfocada a optimizar recuros u optimizar la detección de mujeres de 23 años o más. Así mismo, las implicaciones éticas podrían variar según sea el uso que se da a los resultados.
  + **EDA** 
    Los datos obtenidos provienen de los registros de mujeres que se practican ILE en la CDMX, donde la opción de ejercerla es libre (por cualquier motivo que considere la persona). Existen estados donde la ILE es legal pero sólo bajo ciertos motivos (como violación, que ponga en peligro de la mujer, etc). Se observo que hay mujeres de otros entidades que vienen a la CDMX para practicarse el ILE, eso podría sesgar la motivación o tipo de personas que se practican el ILE en la CDMX.
  + **Imputación de datos:**
    La corrección y/o imputación de valores sugiere que la forma de recabar los datos podría no ser del todo confiable o fidedigna, pues variables como el parentesco con el acompañante, podrían sugerir que las pacientes o recolectores de datos no proporcionan o reportan todos los datos correctamente (existen muchas mujers menores de edad que no reportan parentescon del acompañante, desconociendo si no hubo acompañante o deliverdamente no se reporto).
    Para efectos de poder correr los modelos, para algunas de las variables se sustituyo los NAs con la mediana, sin embargo parala variable nile (no. de ILE) había 20% de faltantes, lo cual podría sesgar la interpretabilidad o efecto en el modelo, si se toma esta variable
  + **Random Forest para selección de variables**
     Para variables categoricas como ocupacion, anticonceptivo, nivle_edu y edocivil_descripcion se probó reducir el numero de categorías que tenían para logara incrementar el valor predictivo, no obstante implica pérdida de información que aporta la variable original. Para el entendimiento del modelo o interpretación, la reducción de información podría derrivar en generalidades imprecisas.
  + **Selección de variables:**
     Para lograr meter variables categóricas a los modelos, se crearon variables dummy, y posterioremente se decidió qué variables se quedarían en el modelo. El seleccionar sólo algunas dummies de una variable categorica (sin abarcar todos los valores que toma), tiene un efecto similar al planteado en feature engineering (con la reducción de categorias).
  +  **Hiperparámetros del modelo (Magic Loop)**
      La selección implicó el probar muchos parámetros para los diferentes modelos probados, escogiendo el que minimizó el score F1, sin indicar las implicaciones y/o diferencias entre uno u otro.